In [ ]:
import numpy as np
import scipy
from scipy import optimize
from scipy.optimize import minimize, dual_annealing
import matplotlib.pyplot as plt
import runModel
import parameters as p
import runModel
import pandas as pd
from tqdm import tqdm #for progress meter
import math
import datetime

df = pd.read_csv('Data.csv')#change name of file if importing a different dataset
    #open and read file
gen = df.loc[(df.Genotype == 'BRON') & (df.Experiment == 1)]#change Genotype name if using different genotype #experiment signifier added because the original dataset contained data from various experiments
#creates data frame for the TEST_VARN genotype from experiment 1
gen = gen.drop_duplicates(subset=['Timepoint_Cold','Temperature','Timepoint'])
#removes any duplicate rows
gen = gen.sort_values("Timepoint")
#presents data in data frame by ascending Timepoint
timetemp = (np.array([gen['Timepoint_Cold'],gen['Temperature'],gen['Timepoint']]))
#makes the data frame into an array containing the [Timepoint_Cold], [Temperature], [Timepoint] data

#VIN3 parameter definitions
p.dV = 9.2
p.dB = 0.00316
p.A1 = 2
p.C1 = 0.0301995
p.D1 = 2.2

daysCold_all= timetemp[0,1:]
#creates array of all cold treatment lengths (days kept at cold temperature) except from the first one (initial conditions)
temp_all= timetemp[1,1:]
#creates array of all temperatures at times of sampling except from the first one (initial conditions)
days_all= timetemp[2,1:]
#creates array of all times of sampling (days since start of cold treatment) except from the first one (initial conditions)
timeTC = set() #keeps unique values, not ordered
#creates empty set timeTC

for i in range(len(daysCold_all)):
        days=days_all[(daysCold_all==daysCold_all[i]) & (temp_all==temp_all[i])] #adds all days where both cold duration and temperature match to array
        temp=temp_all[i]
        daysCold=daysCold_all[i]
        timeTC.add((tuple(days), temp, daysCold))#adds the loop values to the set
        #populates timeTC set with all combinations of days_all, temp_all and daysCold_all except from the initial conditions


#sets up initial VARN values and normalises them to initial Col FRI values
FLC_all = []
#sets up empty array
COLFRIinit = df.loc[(df.Genotype == 'CF') & (df.Timepoint == 0) & (df.Timepoint_Cold == 0) & (df.Experiment == 1)] #The genotype name used should import initial Col FRI data from the used dataset
#imports initial TEST_CF data (before the cold treatment) for experiment 1
CFinit = np.array(COLFRIinit.FLC)
#creates array of CFinit FLC values (initial FLC values before cold treatment)
df = df.loc[(df.Genotype == 'BRON') & (df.Experiment == 1)] #change Genotype name if using different genotype
df.FLC = df.FLC / np.mean(CFinit)
#standardises VARN FLC values in data frame to mean initial Col FRI FLC value
VARNin=df.loc[(df.Genotype == 'BRON') & (df.Timepoint == 0) & (df.Timepoint_Cold == 0) & (df.Experiment == 1)] #change Genotype name if using different genotype
#creates data frame for initial VARN FLC values (when it hasn't beeen put in the cold)
VARNin_flc=np.array(VARNin.FLC)
#creates array of initial VARN FLC values
VARNin_TT=np.array(VARNin.Timepoint)
#creates an array of initial VARN timepoints (should be 0)           
#we now have initial conditions set up and all possible combinations of experimental data defined



def run_model_SSE(x):
    #####FLC MODEL
    
    parameters = [x[0],x[1],p.param[2], p.param[3],x[2],x[3],p.param[6],x[4],p.param[8],p.param[9],p.param[10],p.param[11],p.param[12],x[5]]
    
    try:                          
        SSE=0 #resetting SSE variable to 0 for the upcoming model run
        for entry in timeTC: #for test set, there is only 1 entry so only runs the loop once
            days=np.array(entry[0])
            #set days array to all the days(timepoints) present in the set
            temp=entry[1]
            #sets temp to temp defined in the set
            daysCold=entry[2]
            #sets cold treatment length to the cold treatment length defined in the set
                                                                                                

            tiTi=np.array([[-5,timetemp[0,0],timetemp[0,0]+0.01, daysCold, daysCold+0.01, 100],[22,timetemp[1,0],temp, temp, temp,22]])
            #creates an array with values [time at which temperature occurs][temperature values corresponding to the times] 
            #defines the times and temperatures for changeover between the cold and warm conditions, and also for initial and end points of the model(model starts at -5 days and ends at 100 days)
            p.tiTi=tiTi
            #sets parameter value to tiTi array
            TT=np.array([tiTi[0][0],0,*days, tiTi[0][-1]])#[-1] just means last entry, * just means that it can have an arbritrary number of arguments
            #makes an array of timepoints [-5(start of model),0,all days for which there are data,100(end of model)]
            VIN3,FLC,TT,y=runModel.run_model(parameters,p.tiTi,TT)
            #runs model for experimental data and initial and end timepoints
            FLC_all.append(FLC)
            #appends experimental FLC value generated from model to array to store it
            FLCex_flc = []
            FLCex_TT = []
            for i in range(len(days)): #for every timepoint for which there is experimental data, compares the experimental values with the model generated values
                gen = df.loc[(df.Genotype == 'BRON') & (df.Timepoint == days[i]) & (df.Temperature == temp) & (df.Timepoint_Cold == daysCold) & (df.Experiment == 1)] #change Genotype name if using different genotype
                #creates data frame for day being run
                FLC_ex = np.array(gen.FLC)
                #creates array containing experimental FLC values
                FLCex_flc.extend(list(FLC_ex))
                #adds the elements from FLC_ex to the FLCex_flc list, creates permanent copy of experimental data?
                sumsq = (FLC_ex - FLC[i+2])*(FLC_ex - FLC[i+2])
                #squares the difference between the experimental data point for this specific timepoint and the data point predicted by the model
                SSE = SSE + sumsq.sum()
                #adds all the square differences together and adds this to the already established SSE value for the other days (timepoints) in this run
                FLCex_TT.extend([days[i]]*len(FLC_ex))
                #adds timepoints corresponding to experimental FLC values in the FLCex_flc array
                                                                                                
                                                                                                                                                
            p.tiTi=np.array([[-5,-0.01,0,daysCold,daysCold+0.01,100],[22,22,temp,temp,22,22]])
            #resets tiTi array parameter to define temperatures of the changeover points and edges of the model 
            TT=np.arange(p.tiTi[0][0], p.tiTi[0][-1], 0.25)
            #creates an array of timepoints from -5 to 100 in 0.25 increments to plot the graph on
            VIN3,FLC,TT,y=runModel.run_model(parameters,p.tiTi,TT)
             #runs model for all the increments to produce the curved line graph that is shown                                          
                                                                                  
        gen = df.loc[(df.Genotype == 'BRON') & (df.Timepoint == 0) & (df.Timepoint_Cold == 0) & (df.Experiment == 1)] #change Genotype name if using different genotype
        FLC_exi = np.array(gen.FLC)
        #adds initial FLC values (no cold treatment has occurred) for that experiment into an array

        sumsq = (FLC_exi - FLC[1])*(FLC_exi - FLC[1])
        SSE = SSE + sumsq.sum()
        print(x)
        print(SSE)
        print()
        return SSE
    
    except ValueError:
         print("Value Errors:")
         print(x)
         SSE = float('inf')
         print(SSE)
         print()
         return SSE
         
    
    


#x = [s1,s2,r1,r2,n01,f] #order in which variables are defined in x array
bounds = [(1e-10,1), (1e-10,1),(1e-10,1),(1e-10,1),(1e-10,1),(1e-10,10)]
x0 = [ 0.016875, 0.005875, 0.0171875, 0, 0.178125, 6.765625]

#remove "#"'s to use optimisation method

##DUAL ANNEALING OPTIMISATION
#results['DA'] = optimize.dual_annealing(run_model_SSE, bounds)
#try:
    #results['DA']
#except:
    #try:
        #print(results['DA'])
    #except:
        #try:
              #print(results)
        #except:
            #print("error printing DA output")
#try:
    #np.savetxt("output_dual_annealing.csv",results['DA'],delimiter=",")
#except:
     #print("error saving DA output to file")



##NELDER-MEAD OPTIMISATION#####
res = minimize(run_model_SSE, x0, method='nelder-mead')
print(res)
try:
    np.savetxt("output_nelder_mead.csv",res,delimiter=",")
except:
     print("error saving Nelder-Mead output to file")


###BASIN-HOPPING OPTIMISATION###
#result = scipy.optimize.basinhopping(run_model_SSE, x0, stepsize = 0.025)
#print(result)
#try:
    #np.savetxt("output_basin_hopping.txt",result)
#except:
     #print("Error saving to file")



